In [1]:
MODEL_NAME = "tokyotech-llm/Swallow-13b-instruct-hf"
MODEL_BASE_NAME = MODEL_NAME.split("/")[-1]
LORA_DIR = f"./pretrained_lora_v2_{MODEL_BASE_NAME}"

OUTPUT_MERGED_DIR = f"./tmp_merged_v2_{MODEL_BASE_NAME}"
OUTPUT_QUANTIZED_DIR = f"./pretrained_gptq_v2_{MODEL_BASE_NAME}"

In [2]:
from peft import PeftModel  # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

if not os.path.exists(OUTPUT_MERGED_DIR):
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
    )
    model = PeftModel.from_pretrained(base_model, LORA_DIR)
    model = model.merge_and_unload().half()
    model.save_pretrained(OUTPUT_MERGED_DIR)
    del model  # unload
    del base_model  # unload
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    # save to OUTPUT_SAVE_DIR
    tokenizer.save_pretrained(OUTPUT_MERGED_DIR)
else:
    tokenizer = AutoTokenizer.from_pretrained(OUTPUT_MERGED_DIR)

/home/hotchpotch/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 6/6 [00:51<00:00,  8.55s/it]


In [3]:
DEVICE = "cuda:0"
RESPONSE_MESSAGE = "応答"
RESPONSE_PROMPT = f"### {RESPONSE_MESSAGE}:"

PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"
    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}


def build_prompt(
    user_message: str,
    inputs: str | None = "",
) -> str:
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(
            instruction=user_message, input=inputs
        )
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=user_message)

In [4]:
import pandas as pd
import datasets

ds = datasets.load_dataset("hotchpotch/jaqket_v1_qa_wikija_context")  # type: ignore
train_ds = ds["train"]  # type: ignore
train_df = train_ds.data.to_pandas()  # type: ignore
# context は list なので、 "\n" で結合する
train_df["context"] = train_df["context"].apply(lambda x: "\n".join(x) + "\n")
train_df.head(1)

,qid,question,answer,context,answers,competition,timestamp,section,number,original_question,original_answer,original_additional_info
0,QA20CAPR-0004,『non・no』『週刊プレイボーイ』『週刊少年ジャンプ』といえば、発行している出版社はどこで...,集英社,集英社 株式会社集英社(しゅうえいしゃ)は、日本の総合出版社。『週刊少年ジャンプ』『週刊プレ...,[集英社],第1回AI王,2019/12/25,開発データ問題 (dev1),4,『non・no』『週刊プレイボーイ』『週刊少年ジャンプ』といえば、発行している出版社はどこで...,集英社,


In [5]:
prompts = []
for _, row in train_df.iterrows():
    prompts.append(build_prompt(row["question"], row["context"]) + "\n" + row["answer"])
prompts[0]

'以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。\n\n### 指示:\n『non・no』『週刊プレイボーイ』『週刊少年ジャンプ』といえば、発行している出版社はどこでしょう?\n\n### 入力:\n集英社 株式会社集英社(しゅうえいしゃ)は、日本の総合出版社。『週刊少年ジャンプ』『週刊プレイボーイ』『non-no』『すばる』 『Myojo』などの雑誌を発行している。社名は「英知が集う」の意味。\nNon-no 『平凡パンチ』(平凡出版、現・マガジンハウス)の対抗馬として集英社より『週刊プレイボーイ』が創刊された経緯と同じく、1971年、平凡パンチ女性版(後の『an・an』)に対抗する形で創刊された。発売日は、長らく毎月5日・20日の月2回であったが、2010年9月18日発売の11月号より毎月20日のみとなった。女性ファッション誌の老舗として、1970年代には『an・an』とともに旅行特集を掲載し、アンノン族と呼ばれる、ファッション雑誌やガイドブックを片手に一人旅や少人数で旅行する若い女性を生み出した。人気ファッションモデルを数多く輩出し、女優、タレントに転身し成功した例も数多い。直接のつながりはないが、『Seventeen』より多少ターゲットの年齢が高いという点は創刊以来一貫している。2022年現在の編集長は俵理佳子。\n小説NON 小説NON(しょうせつノン)は、株式会社祥伝社が発行している月刊の小説誌である。1986年6月創刊。毎月22日に発売。判型はA5。雑誌コードは4765。月刊小説誌には、他に『オール讀物』『月刊ジェイ・ノベル』『小説現代』『小説新潮』『小説すばる』『小説宝石』『小説 野性時代』などがある。\n\n\n### 応答:\n集英社'

In [6]:
def get_examples(texts, n_samples=128):
    # https://github.com/PanQiWei/AutoGPTQ/blob/main/examples/quantization/quant_with_alpaca.py
    # では128サンプルを使っているため、ここでも128サンプルを使う
    texts = texts[:n_samples]
    for text in texts:
        yield tokenizer(text)  # type: ignore


examples = get_examples(prompts)

In [7]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig  # type: ignore

quantize_config = BaseQuantizeConfig(
    bits=4,  # quantize model to 4-bit
    group_size=128,  # it is recommended to set the value to 128
    desc_act=False,  # set to False can significantly speed up inference but the perplexity may slightly bad
)
if not os.path.exists(OUTPUT_QUANTIZED_DIR):
    model = AutoGPTQForCausalLM.from_pretrained(OUTPUT_MERGED_DIR, quantize_config)
    model.quantize(examples)
    model.save_quantized(OUTPUT_QUANTIZED_DIR, use_safetensors=True)
    tokenizer.save_pretrained(OUTPUT_QUANTIZED_DIR)

Loading checkpoint shards: 100%|██████████| 6/6 [00:04<00:00,  1.21it/s]
INFO - Start quantizing layer 1/40
INFO - Quantizing self_attn.k_proj in layer 1/40...
INFO - Quantizing self_attn.v_proj in layer 1/40...
INFO - Quantizing self_attn.q_proj in layer 1/40...
INFO - Quantizing self_attn.o_proj in layer 1/40...
INFO - Quantizing mlp.up_proj in layer 1/40...
INFO - Quantizing mlp.gate_proj in layer 1/40...
INFO - Quantizing mlp.down_proj in layer 1/40...
INFO - Start quantizing layer 2/40
INFO - Quantizing self_attn.k_proj in layer 2/40...
INFO - Quantizing self_attn.v_proj in layer 2/40...


INFO - Quantizing self_attn.q_proj in layer 2/40...
INFO - Quantizing self_attn.o_proj in layer 2/40...
INFO - Quantizing mlp.up_proj in layer 2/40...
INFO - Quantizing mlp.gate_proj in layer 2/40...
INFO - Quantizing mlp.down_proj in layer 2/40...
INFO - Start quantizing layer 3/40
INFO - Quantizing self_attn.k_proj in layer 3/40...
INFO - Quantizing self_attn.v_proj in layer 3/40...
INFO - Quantizing self_attn.q_proj in layer 3/40...
INFO - Quantizing self_attn.o_proj in layer 3/40...
INFO - Quantizing mlp.up_proj in layer 3/40...
INFO - Quantizing mlp.gate_proj in layer 3/40...
INFO - Quantizing mlp.down_proj in layer 3/40...
INFO - Start quantizing layer 4/40
INFO - Quantizing self_attn.k_proj in layer 4/40...
INFO - Quantizing self_attn.v_proj in layer 4/40...
INFO - Quantizing self_attn.q_proj in layer 4/40...
INFO - Quantizing self_attn.o_proj in layer 4/40...
INFO - Quantizing mlp.up_proj in layer 4/40...
INFO - Quantizing mlp.gate_proj in layer 4/40...
INFO - Quantizing mlp.do